In [2]:
import os
import time
from tqdm.notebook import tqdm, trange
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
print(torchvision.__version__) # 0.14.1
import torchvision.transforms as transforms
from torchvision.models import resnet34, ResNet34_Weights, resnet18, ResNet18_Weights,\
                                mobilenet_v3_small, MobileNet_V3_Small_Weights


0.14.1+cu116


In [3]:
!pip3 install coremltools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.2 MB/s eta 0:00:00


In [4]:
class ResNetClassifier(nn.Module):
    def __init__(self, output_dim, which_resnet="resnet34"):
        super(ResNetClassifier, self).__init__()
        if which_resnet == "resnet34":
            self.model = resnet34(weights=ResNet34_Weights.DEFAULT)
        elif which_resnet == "resnet18":
            self.model = resnet18(weights=ResNet18_Weights.DEFAULT)
        elif which_resnet == "mobilenet_v3_small":
            self.model = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.DEFAULT)
        else:
            raise NotImplementedError()

        if "resnet" in which_resnet:
            self.model.fc = nn.Linear(self.model.fc.in_features, output_dim)
        elif "mobilenet" in which_resnet:
            self.model.classifier[3] = nn.Linear(self.model.classifier[3].in_features, output_dim)

    def forward(self, img):    
        return self.model(img)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
model_path = "/content/drive/MyDrive/Colab Notebooks/CMU/Capstone/checkpoints/mobilenet3_03-16_08-35/model"
torch_model = torch.load(model_path, map_location=torch.device('cpu'))

In [ ]:
torch_model.eval()

In [24]:
# torch_model = torch_model.eval()
# # with torch.no_grad():
# #     out = torch_model(torch.rand(1, 3, 224, 224))
# out = torch_model(torch.rand(1, 3, 224, 224, requires_grad=False))

# out

In [22]:
example_input = torch.rand(1, 3, 224, 224)
traced_model = torch.jit.trace(torch_model, example_input)
out = traced_model(example_input)

In [23]:
out

tensor([[ 0.2479,  0.9860,  0.0389,  0.3267, -1.0722, -1.5133, -0.5798, -1.5802,
         -0.3619, -0.0049,  0.0886]], grad_fn=<AddmmBackward0>)

In [29]:
import coremltools as ct

# Using image_input in the inputs parameter:
# Convert to Core ML program using the Unified Conversion API.
# model = ct.convert(
#     traced_model,
#     convert_to="mlprogram",
#     inputs=[ct.TensorType(shape=example_input.shape)]
#  )
mlmodel = ct.convert(
    traced_model,
    convert_to="mlprogram",
    inputs=[ct.TensorType(shape=example_input.shape, name='input_image', dtype=np.float32)],
    outputs=[ct.TensorType(name='output_logits', dtype=np.float32)],
)

Running MIL Clean up passes: 100%|██████████| 11/11 [00:01<00:00,  7.14 passes/s]


In [28]:
# model = ct.convert(
#     traced_model,
#     inputs=[ct.TensorType(shape=example_input.shape)]
#  )

In [31]:
## test mlmodel
results = mlmodel.predict({"input_image": example_input.numpy()})
ml_result = results['output_logits'][0]
ml_result

Exception: ignored

In [33]:
mlmodel.save("/content/drive/MyDrive/Colab Notebooks/CMU/Capstone/checkpoints/mobilenet3_03-16_08-35/modelorigami.mlmodel")

Exception: ignored